In [1]:
%%init_spark
launcher.master="yarn"
launcher.num_executors=6
launcher.executor_cores=2
launcher.executor_memory="1600m"

## Problem 1

Customize record delimiter in spark

In [2]:
val conf=sc.hadoopConfiguration
conf.set("textinputformat.record.delimiter", "\n\n")

import org.apache.hadoop.mapreduce.lib.input.TextInputFormat
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text

val wikilinks=sc.newAPIHadoopFile("hdfs://bd-hm:9000/hadoop-user/data/data-00007-of-00010",
                                  classOf[TextInputFormat], classOf[LongWritable], classOf[Text], conf)
                                    .map{case(offset,value)=>value.toString.trim()}
                                    .filter(value=>value.length!=0)

wikilinks.take(2).foreach(println)

Intitializing Scala interpreter ...

Spark Web UI available at http://bd-hm:8088/proxy/application_1571094080322_0007
SparkContext available as 'sc' (version = 2.4.4, master = yarn, app id = application_1571094080322_0007)
SparkSession available as 'spark'


URL	ftp://194.44.98.42/UPLOAD/SOFT/delphi/AForge.NET%20Framework-2.2.3/Release/AForge.Imaging.xml
MENTION	Bayer filter	863044	http://en.wikipedia.org/wiki/Bayer_filter
TOKEN	bluring	905997
TOKEN	well	863642
TOKEN	makes	863424
TOKEN	DarkKhaki	837410
TOKEN	user	863302
TOKEN	enumeration	923873
TOKEN	elemement	849381
TOKEN	gives	847098
TOKEN	diffR	915725
TOKEN	skip	846891
URL	ftp://202.38.89.18/incoming/ASME/data/pdfs/trk-18/IMECE2011-63321.pdf
MENTION	starter	14667	http://en.wikipedia.org/wiki/Starter_motor
TOKEN	position	19094
TOKEN	up	19610
TOKEN	inherently	50290
TOKEN	potion	87557
TOKEN	away	80271
TOKEN	accomplished	26223
TOKEN	over	117210
TOKEN	Emerging	157103
TOKEN	depending	118387
TOKEN	present	144887


conf: org.apache.hadoop.conf.Configuration = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, __spark_hadoop_conf__.xml
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
wikilinks: org.apache.spark.rdd.RDD[String] = MapPartitionsRDD[2] at filter at <console>:35


#### Question 1. Create (URL1, URL2)

In [3]:
//Get a line per URL and Mention/Token
val pairs= wikilinks.map(line=>(line.split("\n")(0), line)).flatMapValues(l=>l.split("\n"))
//Filter to only the URLs and Mentions
val urlMent= pairs.filter((k) => k._2.contains("MENTION"))

urlMent.take(5).foreach(println)

(URL	ftp://194.44.98.42/UPLOAD/SOFT/delphi/AForge.NET%20Framework-2.2.3/Release/AForge.Imaging.xml,MENTION	Bayer filter	863044	http://en.wikipedia.org/wiki/Bayer_filter)
(URL	ftp://202.38.89.18/incoming/ASME/data/pdfs/trk-18/IMECE2011-63321.pdf,MENTION	starter	14667	http://en.wikipedia.org/wiki/Starter_motor)
(URL	ftp://212.154.195.122/Obshaya%20medicina/5%20kurs/Nevrol.Psyhiatr.Narkolog/angl/Lekcii/26.doc,MENTION	physical	27168	http://en.wikipedia.org/wiki/Physical)
(URL	ftp://212.154.195.122/Obshaya%20medicina/5%20kurs/Nevrol.Psyhiatr.Narkolog/angl/Lekcii/26.doc,MENTION	psychological	27384	http://en.wikipedia.org/wiki/Psychological)
(URL	ftp://212.154.195.122/Obshaya%20medicina/5%20kurs/Nevrol.Psyhiatr.Narkolog/angl/Lekcii/26.doc,MENTION	withdrawal	27943	http://en.wikipedia.org/wiki/Withdrawal)


pairs: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[4] at flatMapValues at <console>:30
urlMent: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[5] at filter at <console>:32


In [4]:
//Remove everything except the webpage links
val urlMentClean= urlMent.map((k) => (k._1.split("\t").last, k._2.split("\t").last))
urlMentClean.take(5).foreach(println)

(ftp://194.44.98.42/UPLOAD/SOFT/delphi/AForge.NET%20Framework-2.2.3/Release/AForge.Imaging.xml,http://en.wikipedia.org/wiki/Bayer_filter)
(ftp://202.38.89.18/incoming/ASME/data/pdfs/trk-18/IMECE2011-63321.pdf,http://en.wikipedia.org/wiki/Starter_motor)
(ftp://212.154.195.122/Obshaya%20medicina/5%20kurs/Nevrol.Psyhiatr.Narkolog/angl/Lekcii/26.doc,http://en.wikipedia.org/wiki/Physical)
(ftp://212.154.195.122/Obshaya%20medicina/5%20kurs/Nevrol.Psyhiatr.Narkolog/angl/Lekcii/26.doc,http://en.wikipedia.org/wiki/Psychological)
(ftp://212.154.195.122/Obshaya%20medicina/5%20kurs/Nevrol.Psyhiatr.Narkolog/angl/Lekcii/26.doc,http://en.wikipedia.org/wiki/Withdrawal)


urlMentClean: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[6] at map at <console>:30


#### Question 2. Number of times URL has been mentioned, and 10 most frequently mentioned

In [5]:
//Number of times URL has been mentioned by another URL - in the form (URL, # mentions)
val countMent= urlMentClean.map(word=>word._2.split(",")).map(s=>(s(0).toString)).flatMap(x=>x.split("\n")).map(word=>(word,1)).reduceByKey((v1,v2)=>v1+v2)
countMent.take(25).foreach(println)

(http://en.wikipedia.org/wiki/Dimension_(disambiguation),1)
(http://en.wikipedia.org/wiki/File:Williamson_clint_swci_official_196.jpg,1)
(http://en.wikipedia.org/wiki/Splint_(medical),1)
(http://en.wikipedia.org/wiki/Milbridge,1)
(http://en.wikipedia.org/wiki/Moravian_church,1)
(http://en.wikipedia.org/wiki/Tofik_Bakhramov,2)
(http://en.wikipedia.org/wiki/Thrikodithanam,1)
(http://en.wikipedia.org/wiki/One_Day_at_a_Time,7)
(http://en.wikipedia.org/wiki/Unknot,1)
(http://en.wikipedia.org/wiki/Bourbon_Street,31)
(http://en.wikipedia.org/wiki/Valerie_Plame_affair,1)
(http://en.wikipedia.org/wiki/Small_world_networks,1)
(http://en.wikipedia.org/wiki/Robert_Ervin_Howard,1)
(http://en.wikipedia.org/wiki/Matteo_Nigetti,1)
(http://en.wikipedia.org/wiki/Atom_Egoyan,6)
(http://en.wikipedia.org/wiki/Besud,1)
(http://en.wikipedia.org/wiki/Double-ender,1)
(http://en.wikipedia.org/wiki/Steady_On_(Shawn_Colvin_album),1)
(http://en.wikipedia.org/wiki/Ilgachuz_Range,2)
(http://en.wikipedia.org/wiki/Nos

countMent: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[11] at reduceByKey at <console>:30


In [6]:
//Print top 10 most mentioned urls
val top10=countMent.sortBy(x=>x._2, false)
top10.take(10).foreach(println)

(http://en.wikipedia.org/wiki/Flickr,8009)
(http://en.wikipedia.org/wiki/Wikipedia:Stub,5579)
(http://en.wikipedia.org/wiki/United_States,4551)
(http://en.wikipedia.org/wiki/World_War_II,3148)
(http://en.wikipedia.org/wiki/India,3094)
(http://en.wikipedia.org/wiki/God,2875)
(http://en.wikipedia.org/wiki/Copyright,2178)
(http://en.wikipedia.org/wiki/New_York_City,2063)
(http://en.wikipedia.org/wiki/Europe,2040)
(http://en.wikipedia.org/wiki/U.S._state,1918)


top10: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[16] at sortBy at <console>:30


#### Question 3. Symmetric URL pairs

In [7]:
//Swap Mentions as the key, and URL as value
val mentUrl = urlMentClean.map(x=>(x._2, x._1))

//Join by keys
val join= mentUrl.join(urlMentClean)

join.take(1).foreach(println) 

mentUrl: org.apache.spark.rdd.RDD[(String, String)] = MapPartitionsRDD[17] at map at <console>:30
join: org.apache.spark.rdd.RDD[(String, (String, String))] = MapPartitionsRDD[20] at join at <console>:33


There is no data in the output, so it appears that there are no symmetric URLs in the dataset!